<a href="https://colab.research.google.com/github/MarisaRies/covir/blob/main/COVIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#Encryption
#!pip install cape_privacy

In [27]:
#Lade Bibliotheken
import sqlite3            as sq
from sqlite3 import Error
import pandas             as pd
import numpy              as np
import requests
import datetime           as dt
import seaborn            as sns
import matplotlib.pyplot  as plt
import matplotlib         as mpl
%matplotlib inline

In [28]:
#Pfad Definitionen
drive_path        = "/content/drive"
project_root_path = f"{drive_path}/MyDrive/Studium/COVIR"
image_path        = f"{project_root_path}/Bilder/"
git_path          = "https://raw.githubusercontent.com/MarisaRies/covir/main/"

#Lokaler CSV-Pfad (privater Zugriff) 
csv_file_path     = f"{project_root_path}/CSV/"
#Globaler CSV-Pfad (öffentlicher Zugriff) 
#csv_file_path = '/content/gdrive/Shareddrives/Material/csv_data/covid19/'

In [29]:
#
# Aufbau der Quelldatenbank
#

#Verbinden von GDrive
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

#Verbindung zu SQLite
def create_connection_memory():
  conn_src = None;
  try:
    #Verbindungsaufbau
    conn_src = sq.connect(':memory:')
    return conn_src
    print(sqlite3.version)
  except Error as e:
    print("Quell-Datenbank: Fehler #1 beim Verbinden zu SQLite", e)

#Lokale Definition der SQLite Verbindung
def create_connection_local(local_path):
  conn_src = None;
  try:
    #Verbindungsaufbau
    conn_src = sq.connect(local_path+'/covir_src.db')
    return conn_src
    print(sqlite3.version)
  except Error as e:
    print("Quell-Datenbank: Fehler #2 beim Verbinden zu SQLite", e) 
conn_src = create_connection_memory()

cur_src = conn_src.cursor()
print("Quell-Datenbank: Verbindung erfolgreich zu SQLite")

#Tabellenschema erzeugen
cur_src.executescript(requests.get(git_path+'sql/1src_db_create_statements.sql').text)

conn_src.commit()

print("Quell-Datenbank: Tabellenschema erfolgreich erstellt")

Mounted at /content/drive
Quell-Datenbank: Verbindung erfolgreich zu SQLite
Quell-Datenbank: Tabellenschema erfolgreich erstellt


In [30]:
#
# ETL-Prozess: Extraktion
#

#CSV-Daten aus Google-Drive in Dateframes laden, relevante Spalten auswählen
df_csv_pat = pd.read_csv(csv_file_path+'patients.csv')
df_csv_pat = df_csv_pat[['Id','BIRTHDATE','DEATHDATE','FIRST','LAST']]

df_csv_enc = pd.read_csv(csv_file_path+'encounters.csv')
df_csv_enc = df_csv_enc[['Id','START','STOP','PATIENT','CODE','DESCRIPTION']]

df_csv_con = pd.read_csv(csv_file_path+'conditions.csv')
df_csv_con = df_csv_con[['ENCOUNTER','START','STOP','PATIENT','CODE','DESCRIPTION']]

df_csv_imm = pd.read_csv(csv_file_path+'immunizations.csv')
df_csv_imm = df_csv_imm[['ENCOUNTER','DATE','PATIENT','CODE','DESCRIPTION']]

#Dataframes in Quell-Datenbank laden
df_csv_pat.to_sql('tb_patients', conn_src, if_exists='append', index=False)
df_csv_enc.to_sql('tb_encounters', conn_src, if_exists='append', index=False)
df_csv_con.to_sql('tb_conditions', conn_src, if_exists='append', index=False)
df_csv_imm.to_sql('tb_immunizations', conn_src, if_exists='append', index=False)

conn_src.commit()

#Dataframes löschen (werden nicht mehr gebraucht)
del(df_csv_pat) 
del(df_csv_enc)
del(df_csv_con)
del(df_csv_imm)

print("ETL-Extraktion: Relevante Daten aus CSV extrahiert und in Quelldatenbank eingefügt")

FileNotFoundError: ignored

In [ ]:
#
# ETL-Prozess: Transformation
#

#Tabellenschema für Data-Warehouse aus Quelldatenbank ableiten und temporär in Dataframes speichern
cur_src.execute(
"""
SELECT * 
FROM tb_patients 
"""
)
df_src_pat = pd.DataFrame(cur_src.fetchall(), columns=('Id', 'birthdate', 'deathdate', 'first', 'last'))

cur_src.execute(
"""
SELECT 
  enc.Id,
  enc.start as date, 
  enc.patient,
  enc.code,
  enc.description,
  con.code as action_code,
  con.description as action_desc,
  'con' as action_src
FROM 
  tb_encounters as enc,
  tb_conditions as con
WHERE
(
  con.encounter = enc.Id
)

UNION

SELECT 
  enc.Id,
  enc.start as date, 
  enc.patient,
  enc.code,
  enc.description,
  imm.code as action_code,
  imm.description as action_desc,
  'imm' as action_src
FROM 
  tb_encounters as enc,
  tb_immunizations as imm
WHERE
(
  imm.encounter = enc.Id
);
"""
)
df_src_enc = pd.DataFrame(cur_src.fetchall(), columns=('Id', 'date', 'patient', 'code', 'description', 'action_code','action_desc', 'action_src'))

cur_src.execute(
"""
SELECT DISTINCT 
  code as action_code, 
  description as action_desc 
FROM tb_conditions 

UNION

SELECT DISTINCT 
  code as action_code, 
  description as action_desc 
FROM tb_immunizations 
"""
)
df_src_cod = pd.DataFrame(cur_src.fetchall(), columns=('action_code', 'action_desc'))

print("ETL-Transformation: Schema-Transformation von Quelldatenbank nach Data-Warehouse erfolgreich abgeschlossen")

#Alter berechnen und als neues Feld dem Dataframe für Patienten-Daten hinzufügen
df_src_pat['birthdate'] = pd.to_datetime(df_src_pat['birthdate'], format='%Y-%m-%d') #Daten von String in DateTime umwandeln für spätere Berechnung
df_src_pat['deathdate'] = pd.to_datetime(df_src_pat['deathdate'], format='%Y-%m-%d')

df_src_pat['EOL'] = df_src_pat.deathdate #Temporäre Spalte für Referenzpunkt der Altersberechnung (Todestag oder Heute)
df_src_pat['EOL'] = df_src_pat['EOL'].fillna(pd.to_datetime("today")) #Einsetzen von heutigem Datum, falls kein Todestag bekannt

df_src_pat['age'] =  df_src_pat['EOL'].dt.year - df_src_pat['birthdate'].dt.year #Alter berechnen (Referenztag - Geburtstag) und in neue Spalte 'age' einfügen
df_src_pat = df_src_pat.drop(columns=['EOL']) #Temporäre Spalte wieder löschen

print("ETL-Transformation: Alter berechnet und in Patienten-Tabelle eingefügt")

In [ ]:
#
# ETL-Prozess: Laden
#

#Verbindung zu SQLite
def create_connection_memory():
  conn_dwh = None;
  try:
    #Verbindungsaufbau
    conn_dwh = sq.connect(':memory:')
    return conn_dwh
    print(sqlite3.version)
  except Error as e:
    print("ETL-Laden: Fehler #1 beim Verbinden zu SQLite", e)

#Lokale Definition der SQLite Verbindung
def create_connection_local(local_path):
  conn_dwh = None;
  try:
    #Verbindungsaufbau
    conn_dwh = sq.connect(local_path+'/covir_dwh.db')
    return conn_dwh
    print(sqlite3.version)
  except Error as e:
    print("ETL-Laden: Fehler #2 beim Verbinden zu SQLite", e) 
conn_dwh = create_connection_memory()

cur_dwh = conn_dwh.cursor()
print("ETL-Laden: Verbindung erfolgreich zu SQLite")

#Dataframes in Data-Warehouse laden
df_src_pat.to_sql('tb_patients', conn_dwh, if_exists='replace', index=False)
df_src_enc.to_sql('tb_encounters', conn_dwh, if_exists='replace', index=False)
df_src_cod.to_sql('tb_codes', conn_dwh, if_exists='replace', index=False)

conn_dwh.commit()

#Dataframes löschen (werden nicht mehr gebraucht)
del(df_src_pat) 
del(df_src_enc)
del(df_src_cod)

print("ETL-Laden: Dataframes erfolgreich in Data-Warehouse geladen, Data-Warehouse bereit!")

In [ ]:
#Übersicht Patienten Tabelle
df_dwh_pat = pd.read_sql_query("SELECT * FROM tb_patients", conn_dwh)
df_dwh_pat.info()
df_dwh_pat

In [ ]:
#Grafik Altersverteilung
sns.displot(df_dwh_pat["age"])
#plt.savefig(image_path+'Histogramm_Patientenalter.png')
plt.show()

In [ ]:
#Grafik Alive vs Death
df_dwh_pat["state"] = df_dwh_pat["deathdate"] #Kopieren der Spalte Todestag in neue Spalte "state"
df_dwh_pat["state"] = df_dwh_pat["state"].fillna('Alive') #Normalisierung von NaN nach 'Alive'
df_dwh_pat["state"] = df_dwh_pat["state"].replace(['[^Alive]'], '', regex = True) #Normalisierung jeder Zeile ungleich 'Alive' nach '' mittels regulärem Ausdruck
df_dwh_pat["state"] = df_dwh_pat["state"].replace([''], 'Death') #Ersetzen von '' mit 'Death'

sns.countplot(x="variable", hue= "value", data=pd.melt(df_dwh_pat[["state"]]))
#plt.savefig(image_path+'Histogramm_Lebendig-Tot.png')
plt.show()

In [ ]:
#Übersicht Encounter (Begegnung) Tabelle
df_dwh_enc = pd.read_sql_query("SELECT * FROM tb_encounters", conn_dwh)
df_dwh_enc.info()
df_dwh_enc

In [ ]:
#Grafik Verteilung der Begenungsgründe
df_dwh_enc_dis = pd.read_sql_query("SELECT DISTINCT Id, date, patient, code, description FROM tb_encounters", conn_dwh)
sns.countplot(x="variable", hue= "value", data=pd.melt(df_dwh_enc_dis[["description"]]))
plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
#plt.savefig(image_path+'Histogramm_Begegnungsgrund.png')
plt.show()

In [ ]:
#Grafik zeitliche Verteilung der Begnungen (Jahr)
df_dwh_enc_dis["examination_year"] = pd.DatetimeIndex(df_dwh_enc_dis['date']).year #Normalisierung auf Jahr, um Plot zu verbessern
sns.displot(df_dwh_enc_dis["examination_year"])
#plt.savefig(image_path+'Histogramm_Begegnungsverteilung.png')
plt.show()

In [ ]:
#Grafik zeitliche Verteilung der Begegnungen (2019-2021, monatlich)
#todo

In [ ]:
#Grafik Verteilung der Symptome
df_dwh_con = pd.read_sql_query("SELECT * FROM tb_encounters WHERE action_src = 'con'", conn_dwh)
sns.countplot(y="variable", hue= "value", data=pd.melt(df_dwh_con[["action_desc"]]))
plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
#plt.savefig(image_path+'Histogramm_Symptomverteilung.png')
plt.show()

In [ ]:
#Grafik Anzahl Symptome pro Patient (gestaffelt)
#todo

In [ ]:
#Grafik Verteilung der Impfungen
df_dwh_con = pd.read_sql_query("SELECT * FROM tb_encounters WHERE action_src = 'imm'", conn_dwh)
sns.countplot(y="variable", hue= "value", data=pd.melt(df_dwh_con[["action_desc"]]))
plt.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
#plt.savefig(image_path+'Histogramm_Impfungsverteilung.png')
plt.show()

In [ ]:
#Übersicht Codes Tabelle
df_dwh_pat = pd.read_sql_query("SELECT * FROM tb_codes ORDER BY action_code", conn_dwh)
df_dwh_pat.info()
df_dwh_pat

In [ ]:
# Hypothesen

#
# Geriatrische Patienten (65+) erkranken häufiger an Covid-19.
#

#Grafik Anzahl geriatrischer Patienten (>=65) zu nicht-geriatrischen Patienten (<65)
df_dwh_hyp1_1 = pd.read_sql_query(
    """
    SELECT DISTINCT pat.Id, pat.age
    FROM tb_patients as pat
    """
    , conn_dwh)

df_dwh_hyp1_1['ger_state'] = [">=65" if x >=65 else "<65" for x in df_dwh_hyp1_1['age']] #Umwandlung der Spalte Alter in feste Dimensionen [>=65] und [<65]

sns.displot(df_dwh_hyp1_1["ger_state"])
#plt.savefig(image_path+'Histogramm_geriatrischer_Zustand.png')
plt.show()
print("Anzahl nicht-geriatrischen Patienten (<65) zu geriatrischer Patienten (>=65)")

In [ ]:
#Abfrage aller Patienten mit Alter und Covid-19-Status ('COVID-19' wenn positiv, '' wenn negativ)
df_dwh_hyp1_2 = pd.read_sql_query(
    """
    SELECT DISTINCT(temp.patient), pat.age, temp.desc

    FROM (
      SELECT enc.patient as patient, action_desc as desc
      FROM tb_encounters as enc
      WHERE action_code = '840539006'

      UNION

      SELECT Id, '' as action_desc
      FROM tb_patients
    ) as temp JOIN tb_patients as pat ON pat.Id = temp.patient 
    GROUP BY patient
    """
    , conn_dwh)

df_dwh_hyp1_2["desc"] = df_dwh_hyp1_2["desc"].replace([''], 'Negativ') #Ersetzen von leerem String '' mit dem String 'Negativ'
df_dwh_hyp1_2['age'] = [">=65" if x >=65 else "<65" for x in df_dwh_hyp1_2['age']] #Umwandlung der Spalte Alter in feste Dimensionen [>=65] und [<65]

g = sns.FacetGrid(df_dwh_hyp1_2, col="desc")
g.map(sns.histplot, "age")
#plt.savefig(image_path+'Histogramm_geriatrischer_Zustand.png')
plt.show()
print("Anzahl nicht-geriatrischen Patienten (<65) zu geriatrischer Patienten (>=65), Links für Covid-19-positive Fälle. Rechts für negative Fälle")